In [2]:
import pandas as pd
import numpy as np

Data = pd.read_csv('Salic_Data.csv')
Data.columns = ['IdPRONAC', 'CgcCpf', 'AnoProjeto', 'Sequencial', 'idPlanilhaEtapa', 'UF', 'Municipio', 'Item', 'Unidade', 'Quantidade(Proposta)', 'Ocorrencia(Proposta)', 'QtdeDias(Proposta)', 'ValorUnitario(Proposta)', 'Justificativa(Proposta)', 'Quantidade(Projeto)', 'Ocorrencia(Projeto)', 'QtdeDias(Projeto)', 'ValorUnitario(Projeto)', 'Justificativa(Parecista)', 'Produto']

X = Data[['IdPRONAC', 'Item', 'Quantidade(Proposta)', 'Ocorrencia(Proposta)', 'QtdeDias(Proposta)', 'ValorUnitario(Proposta)', 'Justificativa(Proposta)', 'Produto']]
X.head()

,IdPRONAC,Item,Quantidade(Proposta),Ocorrencia(Proposta),QtdeDias(Proposta),ValorUnitario(Proposta),Justificativa(Proposta),Produto
0,111109,Cartaz,10000.0,1,60,0.52,NaN,Apresentação Musical
1,111638,Locação de computadores,10.0,1,120,1500.00,NaN,Administração do Projeto
2,111348,Projeto de Iluminação,1.0,1,5,20000.00,NaN,Festival/Mostra
3,111729,Coordenador Administrativo,2.0,1,60,2000.00,NaN,Administração do Projeto
4,111643,Maquiador,6.0,1,180,600.00,NaN,Espetáculo de Artes Cênicas


In [31]:
from collections import Counter

products = Counter(X['Produto'])
# products_dict
#for i, j in x.items()

#all_products = [(l,k) for k,l in sorted([(j,i) for i, j in products.items()], reverse=True)]
common_products = products.most_common();
common_products

most_common_products =  common_products[5:15] # -5,-1

# filtered_common_products = []
# for (prod,quant) in most_common_products:
#     prod = X[X['Produto'] == prod]
#     filtered_common_products.extend(prod.values)


# filtered_common_products = []
# for (prod,quant) in most_common_products:
#     prod = X[X['Produto'] == prod]
#     filtered_common_products.extend(prod.values)

# Data_matrix = np.matrix(filtered_common_products)[:,:]

# # A = np.matrix(filtered_common_products)[:,0:6]
# # y = np.matrix(filtered_common_products)[:,7]
# # print(A)
# # print(y)

filter_indices = np.zeros(len(X), dtype=int)
for (prod,quant) in most_common_products:
    prod_array = X['Produto'] == prod
    filter_indices = filter_indices | prod_array


    
X_Selected = X[filter_indices]


featureNameToIndex = {featureName: index for index, featureName in enumerate(X)}

ConjuntoDeItens = list(set(X_Selected['Item']))
ConjuntoDeProjetos = list(set(X_Selected['IdPRONAC']))

# print(ConjuntoDeItens)
# print(ConjuntoDeProjetos)

print(most_common_products)
X_Selected.head()

[('Livro', 52663), ('Exposição de Artes', 43029), ('Modernização e Equipagem de Museus e Espaços Culturais', 39042), ('Festival/Mostra - Audiovisual', 28152), ('Plano Anual', 22712), ('Seminário / Simpósio / Encontro / Congresso / Palestra', 17271), ('Oficina /Workshop/Seminário Audiovisual', 15466), ('Programa de Televisão ', 14335), ('DVD', 14175), ('CD-Áudio ou outro suporte para música', 11476)]


,IdPRONAC,Item,Quantidade(Proposta),Ocorrencia(Proposta),QtdeDias(Proposta),ValorUnitario(Proposta),Justificativa(Proposta),Produto
175,111110,Prova,1.0,50,15,200.0,NaN,Livro
354,121090,Assistente de produção,1.0,2,60,1500.0,NaN,Festival/Mostra - Audiovisual
363,176122,Banner/faixa adesiva/faixa de lona/saia de pal...,21.0,12,15,5.0,NaN,Oficina /Workshop/Seminário Audiovisual
396,111735,Folders,2500.0,1,45,12.5,NaN,Exposição de Artes
397,111652,Edição de Texto,1.0,1,180,6000.0,NaN,Livro


In [32]:
from collections import defaultdict
pronac_itens_dict = defaultdict(list)
X_values = X_Selected.values

for index, item  in enumerate(X_values):
    pronac_itens_dict[item[featureNameToIndex['IdPRONAC']]].append([item[featureNameToIndex['Item']], item[featureNameToIndex['Quantidade(Proposta)']]*item[featureNameToIndex['Ocorrencia(Proposta)']]*item[featureNameToIndex['QtdeDias(Proposta)']]])

In [33]:
pronac_itens_compressed_dict = {}
for key_pronac, value_itens in pronac_itens_dict.items():
    item_value_dict = {}
    for value in value_itens:
        if value[0] in item_value_dict:
            item_value_dict[value[0]] = item_value_dict[value[0]] + value[1]
        else:
            item_value_dict[value[0]] = value[1]
    pronac_itens_compressed_dict[key_pronac] = item_value_dict
    
#pronac_itens_compressed_dict

In [34]:
def buildProjetItemsArray(projectItems, allItems):
    items_arrays = np.zeros(len(allItems))
    for index, item in enumerate(allItems):
        if (item in projectItems):
            items_arrays[index] = 1
    return list(items_arrays)

def retrieveItems_ProjetItemsArray(items_arrays, allItems):
    projectItems = []
    for index, item in enumerate(allItems):
        if(items_arrays[index] == 1):
            projectItems.append(item)
    return list(projectItems)

    items_arrays = np.zeros(len(allItems))
    for index, item in enumerate(allItems):
        if (item in projectItems):
            items_arrays[index] = 1
    return list(items_arrays)
            

projectsItemsMatrix = []
projectKind = []
projectDict = {}

for index, pronacID in enumerate(ConjuntoDeProjetos):  
    x = buildProjetItemsArray(pronac_itens_compressed_dict[pronacID],ConjuntoDeItens)
    projectsItemsMatrix.append(x)
    
    product_type = X_Selected[X_Selected['IdPRONAC'] == pronacID]['Produto'].values[0]

    projectTypeArray = np.zeros(len(most_common_products))
    for i, (prod, num) in enumerate(most_common_products):
        if (prod == product_type):
            projectTypeArray[i]= True
        projectDict[product_type] = projectTypeArray.tolist()
        
            
    ### CREATE DICTIONARY
    
    projectKind.append(projectTypeArray.tolist())


#     # Testing vector
#     print('\n', pronacID, ' - ',product_type,'\n***********\n')
#     for i in range(0, len(ConjuntoDeItens)):
#         if(x[i] == 1):
#             print(ConjuntoDeItens[i])
#     print(pronac_itens_compressed_dict[pronacID])

# #     # Using less data to train faster  
#     if (index == 500):
#         break;
        
        
        
# print('\n***********\n')    
# print(projectsItemsMatrix)
# print(projectKind)


In [35]:
def find_category_name_by_array(category_array):
    for (key, value) in projectDict.items():
        if value == category_array:
            return key


# To list all
def get_category_array(projectKind):
    return [find_category_name_by_array(category_array) for category_array in projectKind]

# To list all
def get_items_array(projectsItemsMatrix,allItems):
    return [retrieveItems_ProjetItemsArray(projectsItems, allItems) for projectsItems in projectsItemsMatrix]





# print(get_items_array(projectsItemsMatrix, ConjuntoDeItens))
# print(get_category_array(projectKind))

In [36]:
import numpy as np
from sklearn.cross_validation import train_test_split


projectsItemsMatrix, projectsItemsMatrix_test_final, projectKind, projectKind_test_final = train_test_split(projectsItemsMatrix, projectKind , test_size=0.1, random_state=13)

print(len(projectsItemsMatrix_test_final))
print(len(projectKind_test_final))

projectsItemsMatrix_train, projectsItemsMatrix_test, projectKind_train, projectKind_test = train_test_split(projectsItemsMatrix, projectKind , test_size=0.25, random_state=13)

print(len(projectsItemsMatrix_train))
print(len(projectsItemsMatrix_test))
print(len(projectKind_train))
print(len(projectKind_test))

548
548
3692
1231
3692
1231


In [59]:

# #projectsItemsMatrix_train
# print(get_items_array(projectsItemsMatrix_train, ConjuntoDeItens))

In [60]:
# #projectKind_train
# print(get_category_array(projectKind_train))

In [61]:
# #projectsItemsMatrix_test
# print(get_items_array(projectsItemsMatrix_test, ConjuntoDeItens))

In [62]:
# #projectKind_test
# print(get_category_array(projectKind_test))

In [37]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(len(most_common_products)*6, activation='relu', input_dim=len(ConjuntoDeItens))) #input_shape=(len(ConjuntoDeItens),)
model.add(layers.Dense(len(most_common_products)*3, activation='relu'))
model.add(layers.Dense(len(most_common_products)*3, activation='relu'))
model.add(layers.Dense(len(most_common_products)*3, activation='relu'))
model.add(layers.Dense(len(most_common_products), activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(np.array(projectsItemsMatrix_train),
          np.array(projectKind_train),
          epochs=50,
          batch_size=512,
          validation_data=(np.array(projectsItemsMatrix_test), np.array(projectKind_test)))

score = model.evaluate(np.array(projectsItemsMatrix_test_final), np.array(projectKind_test_final))

print('Test loss:', score[0])
print('Test accuracy:', score[1])


# prediction = model.predict(np.array(projectsItemsMatrix_test_final))
# print(np.shape(projectsItemsMatrix_test_final))
# print(np.shape(prediction))


#print(prediction)
#get_category_array(np.round(prediction))


Train on 3692 samples, validate on 1231 samples
Epoch 1/50
3692/3692 [==============================] - 0s 77us/step - loss: 2.1965 - acc: 0.3155 - val_loss: 1.9481 - val_acc: 0.4525
Epoch 2/50
3692/3692 [==============================] - 0s 14us/step - loss: 1.7639 - acc: 0.5119 - val_loss: 1.4894 - val_acc: 0.5613
Epoch 3/50
3692/3692 [==============================] - 0s 14us/step - loss: 1.3549 - acc: 0.6200 - val_loss: 1.2075 - val_acc: 0.6824
Epoch 4/50
3692/3692 [==============================] - 0s 14us/step - loss: 1.0988 - acc: 0.7172 - val_loss: 1.0192 - val_acc: 0.7449
Epoch 5/50
3692/3692 [==============================] - 0s 15us/step - loss: 0.9053 - acc: 0.7909 - val_loss: 0.8693 - val_acc: 0.8188
Epoch 6/50
3692/3692 [==============================] - 0s 14us/step - loss: 0.7534 - acc: 0.8529 - val_loss: 0.7819 - val_acc: 0.8351
Epoch 7/50
3692/3692 [==============================] - 0s 14us/step - loss: 0.6450 - acc: 0.8749 - val_loss: 0.7040 - val_acc: 0.8505
Epoch 8

In [67]:
prediction = model.predict(np.array(projectsItemsMatrix_test_final))
print(np.shape(projectsItemsMatrix_test_final))
print(np.shape(prediction))

probabilities = prediction*projectKind_test_final*100
correct_probality = sum(probabilities.T)
# print(probabilities[0:10,:])

print(np.shape(correct_probality))

# for invalid_pred in correct_probality:
#     print(invalid_pred)
  
total_less_than_50 = 0
total_less_than_30 = 0
total_less_than_20 = 0
for index, invalid_pred in enumerate(correct_probality):
    if (invalid_pred < 30.0):
        total_less_than_30 = total_less_than_30 + 1

        print("=============")
        prediction_array = np.zeros(len(prediction[index]))
        prediction_array[np.argmax(prediction[index])] = 1
#         print(projectKind_test_final[index])
#         print(prediction_array.tolist())

        print('prediction:', find_category_name_by_array(prediction_array.tolist()), ' - ' , sum(prediction[index]*prediction_array))
        print('expected prediction: ', find_category_name_by_array(projectKind_test_final[index]), ' - ',invalid_pred)
#         print('prediction acc: ', invalid_pred)
#         print('prediction:', find_category_name_by_array(prediction_array.tolist()))
#         print('expected prediction: ', find_category_name_by_array(projectKind_test_final[index]))
        print("**************")
        print(retrieveItems_ProjetItemsArray(projectsItemsMatrix_test_final[index], ConjuntoDeItens))
        print("=============")
        

        
    if (invalid_pred < 20.0):
        total_less_than_20 = total_less_than_20 + 1
    if (invalid_pred < 50.0):
        total_less_than_50 = total_less_than_50 + 1



print('Pelo menos 50% de equivalencia :', (1 - (total_less_than_50/len(correct_probality)))*100,'%')
print('Pelo menos 30% de equivalencia :', (1 - (total_less_than_30/len(correct_probality)))*100,'%')
print('Pelo menos 20% de equivalencia :', (1 - (total_less_than_20/len(correct_probality)))*100,'%')

(548, 945)
(548, 10)
(548,)
prediction: Exposição de Artes  -  0.9737313389778137
expected prediction:  Plano Anual  -  0.16167804133147
**************
['Curadoria', 'Coordenação geral']
prediction: Exposição de Artes  -  0.872856616973877
expected prediction:  Oficina /Workshop/Seminário Audiovisual  -  0.17195696709677577
**************
['Montagem e desmontagem', 'Coordenador de produção', 'Assessor de imprensa', 'Projeto Gráfico', 'Curador', 'Assistente de curadoria', 'Coordenador do Projeto', 'Assistente de produção', 'Diárias', 'Materiais e equipamentos para montagem', 'Produtor Executivo', 'Palestrante', 'Cartaz', 'Banner/faixa adesiva/faixa de lona/saia de palco/testeira/pórtico\n', 'Confecção de Convites', 'Folders', 'Outdoors', 'Coordenação pedagógica', 'Painéis', 'Som', 'Mídia radiofônica', 'Hospedagem sem Alimentação', 'Locação de auditório', 'Material de apoio pedagógico', 'Locação de Espaço para Exposição']
prediction: CD-Áudio ou outro suporte para música  -  0.7123499512

NUMERO DE PROJETOS -> ACURACIA

    02 -> 0.9686
    03 -> 0.8921
    04 -> 0.8779
    05 -> 0.8179
    ...
    10 -> 0.7542
    ...
    15 -> 0.6876
    ...
    20 -> 0.6716